### Postprocessing `.out` files

In [1]:
import re

In [2]:
def get_lines(filename):
    with open(filename) as f:
        lines = f.readlines()
    return lines

In [3]:
def extract_nums(string):
    remove_zeros = lambda x: x != 0
    lnums = [float(s) for s in re.findall(r'-?\d+\.?\d*', string)]
    return list(filter(remove_zeros, lnums))

In [4]:
def get_accuracy(filename):
    lines = get_lines(filename)
    for i, l in enumerate(lines):
        if 'Accuracy' in l:
            string_acc = ''.join(lines[i:])
            break
    return extract_nums(string_acc)

### Reader for `.out` files deprecated

In [5]:
import os
import pandas as pd
import itertools

In [6]:
def read(directory = 'eval'):
    root = '/'.join([os.getcwd(), directory])
    files = sorted(os.listdir(root))
    accs = [get_accuracy('/'.join([root, f])) for f in files]
    return files, accs

In [7]:
files, accs = read()

In [8]:
ann_snn = []
for i, e in enumerate(accs):
    if i % 2 == 0:
        ann_snn.append(e + accs[i + 1])

In [9]:
model = ['vgg', 'res']
data = ['cifar10', 'cifar100']
ls = [16, 8, 4, 2]

In [10]:
models = ['_'.join([str(s) for s in elem]) for elem in itertools.product(model, data, ls)]

In [11]:
df = pd.DataFrame(ann_snn)
df = df[[0, 1, 3, 7, 15]]
df.columns = ['ann', 't=2', 't=4', 't=8', 't=16']
df.insert(loc=0, column='model', value=models)

In [12]:
df

,model,ann,t=2,t=4,t=8,t=16
0,vgg_cifar10_16,0.9404,0.1114,0.8320,0.9135,0.9343
1,vgg_cifar10_8,0.9230,0.3257,0.7643,0.8665,0.9100
2,vgg_cifar10_4,0.1687,0.1021,0.0921,0.1030,0.1185
3,vgg_cifar10_2,0.1000,0.1000,0.1000,0.1000,0.1000
4,vgg_cifar100_16,0.7280,0.1092,0.2224,0.3822,0.5703
5,vgg_cifar100_8,0.7073,0.0736,0.3365,0.5049,0.6283
6,vgg_cifar100_4,0.6828,0.2027,0.3507,0.4902,0.6002
7,vgg_cifar100_2,0.0100,0.0100,0.0100,0.0100,0.0100
8,res_cifar10_16,0.9578,0.6598,0.8167,0.9099,0.9443
9,res_cifar10_8,0.9543,0.7211,0.8525,0.9221,0.9479


In [14]:
# df.to_csv('logs_results.csv', index=False)

### Reader for `.out` files

In [5]:
import os
import pandas as pd
import itertools

In [14]:
def read(directory):
    root = '/'.join([os.getcwd(), directory])
    files = sorted(os.listdir(root))
    accs = {f[:-4]: get_accuracy('/'.join([root, f])) for f in files}
    return accs

In [15]:
def merge_ann_snn(accs):
    new_accs = {}
    for i, e in enumerate(accs):
        if i % 2 == 0:
            new_accs[e[:-4]] = accs[e]
        else:
            new_accs[e[:-4]].extend(accs[e])
    return new_accs

In [26]:
accs = merge_ann_snn(read(directory='20220511/eval'))

In [27]:
cols = ['ann']
for i in range(64):
    cols.append('t=' + str(i + 1))

In [28]:
df = pd.DataFrame.from_dict(accs, orient='index', columns=cols)
df_short = df[['ann', 't=2', 't=4', 't=8', 't=16', 't=32', 't=64']]

In [29]:
df_short

,ann,t=2,t=4,t=8,t=16,t=32,t=64
resnet18_cifar100_LR01_L02,0.7026,0.6281,0.6627,0.6901,0.7019,0.7041,0.7048
resnet18_cifar100_LR01_L04,0.7331,0.5878,0.6569,0.7045,0.7268,0.7342,0.7351
resnet18_cifar100_LR01_L08,0.7522,0.5151,0.6173,0.6982,0.7373,0.7496,0.7522
resnet18_cifar100_LR01_L16,0.7532,0.3290,0.5246,0.6618,0.7249,0.7454,0.7516
resnet18_cifar100_LR05_L02,0.7509,0.6913,0.7217,0.7427,0.7493,0.7527,0.7512
...,...,...,...,...,...,...,...
vgg16_cifar10_LR10_L16,0.9404,0.7455,0.8738,0.9217,0.9357,0.9386,0.9398
vgg16_cifar10_LR20_L02,0.1000,0.1000,0.1000,0.1000,0.1000,0.1000,0.1000
vgg16_cifar10_LR20_L04,0.1000,0.1000,0.1000,0.1000,0.1000,0.1000,0.1000
vgg16_cifar10_LR20_L08,0.9134,0.6108,0.7842,0.8768,0.9044,0.9119,0.9131


In [30]:
df.to_csv('logs_results_long.csv')
df_short.to_csv('logs_results_short.csv')